# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цели**

Цель исследования - проверить следующие гипотезы:

- семейное положение влияет на факт погашения кредита в срок;
- количество детей клиента влияет на на факт погашения кредита в срок;
- уровень дохода влияет на возврат кредита в срок;
- цель кредита влияет на его возврат в срок.

**План исследования**

- изучить таблицу;
- определить и заполнить пропущенные значения;
- удалить дубликаты;
- выделить леммы в значениях столбца с целями получения кредита;
- категоризировать данные;
- обработка артефактов в данных;
- проверить гипотезы;
- выводы исследования.

## Открываем файл с данными и изучаем общую информацию

In [39]:
#импортируем библиотеки и выведем на экран первые и последние позиции датасета, чтобы оценить его
import pandas as pd
import seaborn as sns
from pymystem3 import Mystem
from collections import Counter
import numpy as np

data = pd.read_csv('/datasets/data.csv')
display(data.head())
display(data.tail())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


Итак, в таблице 12 столбцов. Названия столбцов корректные.

Согласно документации к данным:

- children — количество детей в семье;
- days_employed — общий трудовой стаж в днях;
- dob_years — возраст клиента в годах;
- education — уровень образования клиента;
- education_id — идентификатор уровня образования;
- family_status — семейное положение;
- family_status_id — идентификатор семейного положения;
- gender — пол клиента;
- income_type — тип занятости;
- debt — имел ли задолженность по возврату кредитов;
- total_income — ежемесячный доход;
- purpose — цель получения кредита.

In [40]:
#посмотрим описательную информацию о датасете
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

В каждой строке таблицы мы видим данные о клиенте банка. Два столбца (days_employed и total_income) имеют пропуски. Далее следует отметить, что существуют отрицательные значения в стоблцах children и days_employed, а также нулевые значения в столбце dob_years.
Предварительно можно сказать, что данных достаточно для проверки гипотезы.

**Решим следующие пробелмы**

- обработаем пропуски;
- избавимся от дублирующих столбцов;
- исправим некорректные значения.


## Предобработка данных

### Обработка пропусков

In [41]:
#проверим наличие пропусков в датасете
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Мы видим, что в столбцах общий трудовой стаж и ежемесячный доход одинаковое количество пропусков. Это может иметь место в связи с конфиденциальностью информации (касается категории госслужащих), ее отсутствием, в связи с тем, что клиент работает неофициально или является безработным.

In [42]:
#предварительно заполним пробелы средними значениями по столбцам
days_employed_avg = data['days_employed'].mean()
data['days_employed'] = data['days_employed'].fillna(days_employed_avg)
total_income_avg = data['total_income'].mean()
data['total_income'] = data['total_income'].fillna(total_income_avg)

#проверим корретность заполнения пробелов
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Мы заполнили пропуски в данных, чтобы результат нашего ислледования был как можно точнее.

### Замена типа данных

In [43]:
#заменим вещественный тип данных в столбце days_employed и total_income на целочисленный
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

#проверим результат
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Для удобства воприятия и упрощения дальнейших операций мы заменили тип данных с вещественного на целочисленный, используя метод .astype.

### Обработка дубликатов

In [44]:
#приведем строки к нижнему регистру, а затем посчитаем общее количество дубликатов
#воспользуемся методом .duplicated(), потому что мы ищем дубликаты во всем датасете
columns = ['education', 'family_status', 'gender', 'income_type', 'purpose']
for row in (columns):
    data[row] = data[row].str.lower()
#посчитаем количество дубликатов
data.duplicated().sum()

71

Мы обнаружили 71 дубликат. Они могли возникнуть из-за человеческого фактора при ручном введении параметров в базу данных.

In [45]:
#удалим все дубликаты с изменением индексации
data = data.drop_duplicates().reset_index(drop=True)
#перепроверим количество дубликатов
data.duplicated().sum()

0

Далее проверим на корректность данных каждый столбец отдельно.

In [46]:
#воспользуемся методом .value_counts() для проверки значений
data['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Мы видим, что скорее всего имеет место несколько человеческих ошибок при ручном введении данных. Отрицательного количества детей быть не может, а в наше время наличие двадцати детей крайне маловероятно. Поэтому будем считать, что ноль там приписан случайно.

In [47]:
#удалим второй ноль в двадцати и 'минус' у единицы 
data['children'] = data['children'].abs()
data['children'] = data['children'].replace(20, 2)
#проверим результат
data['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

In [48]:
#переведем общий трудовой стаж в года для удобства и посмотрим на данные
data['years_employed'] = data['days_employed'] // 365
data['years_employed'] = data['years_employed'].astype('int')
sorted(data['years_employed'].unique())

[-51,
 -49,
 -46,
 -45,
 -44,
 -43,
 -42,
 -41,
 -40,
 -39,
 -38,
 -37,
 -36,
 -35,
 -34,
 -33,
 -32,
 -31,
 -30,
 -29,
 -28,
 -27,
 -26,
 -25,
 -24,
 -23,
 -22,
 -21,
 -20,
 -19,
 -18,
 -17,
 -16,
 -15,
 -14,
 -13,
 -12,
 -11,
 -10,
 -9,
 -8,
 -7,
 -6,
 -5,
 -4,
 -3,
 -2,
 -1,
 172,
 900,
 901,
 902,
 903,
 904,
 905,
 906,
 907,
 908,
 909,
 910,
 911,
 912,
 913,
 914,
 915,
 916,
 917,
 918,
 919,
 920,
 921,
 922,
 923,
 924,
 925,
 926,
 927,
 928,
 929,
 930,
 931,
 932,
 933,
 934,
 935,
 936,
 937,
 938,
 939,
 940,
 941,
 942,
 943,
 944,
 945,
 946,
 947,
 948,
 949,
 950,
 951,
 952,
 953,
 954,
 955,
 956,
 957,
 958,
 959,
 960,
 961,
 962,
 963,
 964,
 965,
 966,
 967,
 968,
 969,
 970,
 971,
 972,
 973,
 974,
 975,
 976,
 977,
 978,
 979,
 980,
 981,
 982,
 983,
 984,
 985,
 986,
 987,
 988,
 989,
 990,
 991,
 992,
 993,
 994,
 995,
 996,
 997,
 998,
 999,
 1000,
 1001,
 1002,
 1003,
 1004,
 1005,
 1006,
 1007,
 1008,
 1009,
 1010,
 1011,
 1012,
 1013,
 1014,
 1015,
 10

Здесь мы видим слишком большие значения и отрицательные, которые не могут соответствоать реальности. Однако данные об общем трудовом стаже нам не понадобятся, поэтому оставим их в таком виде.

In [49]:
#проверим столбец с возрастом клиентов в годах
sorted(data['dob_years'].unique())

[0,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75]

Мы видим, что есть нулевые значения. Такая ситуация могла возникнуть либо в связи с технической ошибкой при вводе, либо из-за того, что это поле не обязательно для заполнения. Его данные не понадобятся нам для проверки гипотез, поэтому оставим его без изменений.

In [50]:
data['education'].value_counts()

среднее                15172
высшее                  5250
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

В данном столбце все данные реальны.

In [51]:
data['education_id'].value_counts()

1    15172
0     5250
2      744
3      282
4        6
Name: education_id, dtype: int64

Мы видим, что данные в столбцах education и education_id полностью дублируют друг друга, в связи с чем увеличивается размер файла и время обработки данных. В следующем разделе создадим словарь и оптимизируем данные.

In [52]:
data['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [53]:
data['family_status_id'].value_counts()

0    12339
1     4151
4     2810
3     1195
2      959
Name: family_status_id, dtype: int64

Ситуация со столбцами family_status и family_status_id аналогична.

In [54]:
data['gender'].value_counts()

f      14174
m       7279
xna        1
Name: gender, dtype: int64

Неуказанный пол может быть связан как с человеческой ошибкой, так и с личными взглядами заемщика. Этот показатель не повлияет на анализ данных.

In [55]:
data['income_type'].value_counts()

сотрудник          11084
компаньон           5078
пенсионер           3829
госслужащий         1457
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Все данные корректны.

In [56]:
data['debt'].value_counts()

0    19713
1     1741
Name: debt, dtype: int64

Все данные корректны.

In [57]:
display(data.groupby(['total_income']).sum())

,children,days_employed,dob_years,education_id,family_status_id,debt,years_employed
total_income,,,,,,,
20667,0,359219,57,1,0,1,984
21205,0,369708,37,1,1,0,1012
21367,1,-3642,52,1,0,0,-10
21695,0,359726,68,1,1,0,985
21895,0,346602,61,1,0,0,949
...,...,...,...,...,...,...,...
1711309,0,-5734,42,0,1,0,-16
1715018,0,-4719,61,1,4,0,-13
1726276,1,-5248,35,1,1,0,-15


Мы видим очень большие расхождения в данных, однако люди с разным доходом могут брать кредиты для различных целей. Кроме того, такие большие суммы ежемесячного дохода могут возникать из-за оформления дополнительных (неофициальных) доходов, которые не проверяются банком или государством.

**Вывод**

Мы обнаружили несколько видов проблем в данных. Некоторые мы исправили, потому что нам это было необходимо для проверки гипотез, некоторые оставили без изменения.

### Лемматизация

In [58]:
#так как мы импортировали необходимые библиотеки в начале, то сейчас нам необходимо конвертировать столбец purpose в строку,
#а затем провести лемматизацию строки с русским текстом и с использованием Counter реализовать подсчет неизменяемых лемм

m = Mystem()
lemmas = ', '.join(data['purpose'])
lemmas = m.lemmatize(lemmas)
display(Counter(lemmas))

Counter({'покупка': 5897,
         ' ': 33570,
         'жилье': 4460,
         ', ': 21453,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 853,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'получение': 1314,
         'высокий': 1374,
         'подержанный': 111,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607,
         '\n': 1})

**Вывод**

Проведя лемматизацию целей, мы можем выделить несколько больших категорий для цели кредита и создать новый столбец с оптимизированными значениями:

- недвижимость;
- автомобиль;
- образование;
- свадьба.

### Категоризация данных

In [59]:
#выделим словари для нашего набоа данных, чтобы привести их в удобный и читаемый вид
#классифицируем словари по типу, удалив дублирующие столбцы
data_education = data[['education_id', 'education']]
data_family_status = data[['family_status_id', 'family_status', 'debt']]
#в последний словарь мы добавили столбец debt для удовбства проверки гипотезы в дальнейшем

Так как у нас не стоит задачи определить зависимость возврата кредита от возраста, то классифицировать по возрастной группе мы не будем.

In [60]:
#далее напишем функцию для категоризации целей кредита
#и применим ее к столбцу целей
#затем проверим работу функции
def purpose_tranformation(purpose):
    try:
        if 'свадьб' in purpose:
            return 'свадьба'
        if 'недвижим' in purpose or 'жил' in purpose:
            return 'недвижимость'
        if 'автомобил' in purpose:
            return 'автомобиль'
        if 'образова' in purpose:
            return 'образование'
    except:
        return 'такой цели кредита не обнаружено'
data['purpose_optimized'] = data['purpose'].apply(purpose_tranformation)
data['purpose_optimized'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_optimized, dtype: int64

In [61]:
#также для удобства подсчетов напишем функцию для группировки данных по ежемесячному доходу
#предположим, что значения указаны в рублях
#выделим несколько категорий граждан "до 100т.р.", "от 100т.р. до 300т.р." и "свыше 300т.р."
#затем проверим результат

def total_income_mid(total_income):
    try:
        if total_income <= 100000:
            return 'малый доход'
        if 100000 < total_income <= 300000:
            return 'средний доход'
        if total_income > 300000:
            return 'большой доход'
    except:
        return 'неправильное значение'

data['total_income_middle'] = data['total_income'].apply(total_income_mid)
data['total_income_middle'].value_counts()

средний доход    15509
малый доход       4463
большой доход     1482
Name: total_income_middle, dtype: int64

In [62]:
#создадим новые словари, с учетом прописанных функций
data_purpose = data[['purpose', 'purpose_optimized', 'debt']]
data_total_income = data[['total_income', 'total_income_middle', 'debt']]
data_main = data[['children', 'dob_years', 'education_id', 'family_status_id', 'gender','debt', 'purpose_optimized', 'total_income_middle']]

**Вывод**

Мы создали несколько словарей, удалив дублирующие параметры, оптимизировав стобец целей и дохода. Мы упростили визуальную работу с таблицей, сократили размер файла и время обработки данных, а также упростили фильстрацию по цели. Теперь мы можем приступать к проверке игпотез.

## Ответчаем на поставленные вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [63]:
#для наглядности результата выведем сводную таблицу с отношением должников к общему количеству
#заемщиков по каждой категории в процентах

pivot = data.pivot_table('debt', index='children', aggfunc='mean').reset_index()
pivot['debt'] = round(pivot['debt'] * 100, 2)
pivot

,children,debt
0,0,7.54
1,1,9.17
2,2,9.49
3,3,8.18
4,4,9.76
5,5,0.00


**Вывод**

По данным таблицы мы видим, что в процентном соотношении зависимость между наличием детей и возвратом кредита в срок небольшая. Семьи с пятью детьми вносят платежи всегда в срок, однако количества данных по таким семьям крайне мало. Самый маленький процент должников среди семей, у который детей нет, он равняется 7.54%. Далее идут должники с тремя детьми, их 8.18%. Хуже всего вносят платежи семьи с одним, двумя и четырьмя детьми, их показатели 9.16%, 9.49% и 9.75% соответствено.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [64]:
#для наглядности результата выведем сводную таблицу с отношением должников к общему количеству заемщиков
#по каждой категории в процентах

pivot = data.pivot_table('debt', index='family_status', aggfunc='mean').reset_index()
pivot['debt'] = round(pivot['debt'] * 100, 2)
pivot

,family_status,debt
0,в разводе,7.11
1,вдовец / вдова,6.57
2,гражданский брак,9.35
3,женат / замужем,7.55
4,не женат / не замужем,9.75


**Вывод**

По данным таблицы мы видим, что зависимость между семейным положением и возвратом кредита в срок выражена сильнее. Самый низкий процент задолженностей у вдовцов, далее идут разведенные заемщики и щиемщики в браке. Больше всего задолженностей у тех, кто еще не состоял в браке, почти 10%.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [65]:
#для наглядности результата выведем сводную таблицу с отношением должников к общему количеству заемщиков
#по каждой категории в процентах

pivot = data.pivot_table('debt', index='total_income_middle', aggfunc='mean').reset_index()
pivot['debt'] = round(pivot['debt'] * 100, 2)
pivot

,total_income_middle,debt
0,большой доход,7.15
1,малый доход,7.93
2,средний доход,8.26


**Вывод**

Как мы и предполагали, люди с высоким доходом выплачивают кредиты стабильнее. Однако оказвается, что люди со средним доходом имеют немного больший процент задолженностей, нежели чем люди с малым доходом.

- Как разные цели кредита влияют на его возврат в срок?

In [68]:
#для наглядности результата выведем сводную таблицу с отношением должников к общему количеству заемщиков
#по каждой категории в процентах

data.pivot_table('debt', index='purpose_optimized', aggfunc='mean').reset_index()

,purpose_optimized,debt
0,автомобиль,0.093590
1,недвижимость,0.072334
2,образование,0.092200
3,свадьба,0.080034


**Вывод**

По таблице видно, что люди, берущие средства на покупку автомобилей или на образование склонны немного хуже возвращать их банку. Самыми стабильными являются потребители, выполняющие операции с недвижимостью.

## Общий вывод

Мы проверили четыре гипотезы и выяснили следующее:
- зависимость между наличием детей и возвратом кредита в срок небольшая. Семьи с пятью детьми вносят платежи всегда в срок, однако количества данных по таким семьям крайне мало. Самый маленький процент должников среди семей, у который детей нет, он равняется 7.54%. Далее идут должники с тремя детьми, их 8.18%. Хуже всего вносят платежи семьи с одним, двумя и четырьмя детьми, их показатели 9.16%, 9.49% и 9.75% соответствено.
- Зависимость между семейным положением и возвратом кредита в срок выражена больше всего. Самый низкий процент задолженностей у вдовцов и вдов, он равняется 6,57%. Далее идут заемщики в разводе и в браке, процент их невыплат составляет 7.11% и 7.54% соответственно. Больше всего задолженностей у тех, кто еще не состоял в браке, 9.75%. Заемщики, находящиеся в гражданском браке выплачивают средства хуже среднего, процент просрочек равен 9.35%.
- Зависимсоть между доходом и возвратом кредита в срок наименьшая. Логично, что люди с высоким доходом выплачивают кредиты стабильнее всего, процент просрочек по их платежам составляет 7.15%. Однако, как оказалось, люди со средним доходом имеют немного больший процент задолженностей (8.25%), нежели чем люди с малым доходом (7.93%).
- Зависимсоть между целью кредита и его возвратом в срок небольшая. Люди, берущие средства на покупку автомобилей или на образование склонны немного хуже возвращать их банку, процент их просрочек равен 9.35% и 9.22% соответственно. Самыми стабильными являются потребители, выполняющие операции с недвижимостью, среди них 7.23% должников. Заемщики, имеющие свадебные цели выплачивают кредиты немного хуже, они имеют 8% прочрочек.

In [69]:
#так как в описании было сказано, что нам нужно разобраться, влияет ли семейное положение
#и количество детей клиента на факт погашения кредита в срок,
#для нагляндности построим сводную таблицу этой зависимости

data.pivot_table(index='family_status', columns='children', values='debt', aggfunc=['sum']).reset_index()

family_status    sum                              
children                             0      1      2     3    4    5
0                     в разводе   55.0   21.0    8.0   1.0  0.0  NaN
1                вдовец / вдова   53.0    7.0    3.0   0.0  0.0  NaN
2              гражданский брак  229.0  118.0   33.0   8.0  0.0  0.0
3               женат / замужем  516.0  247.0  148.0  17.0  3.0  0.0
4         не женат / не замужем  210.0   52.0   10.0   1.0  1.0  NaN